In [ ]:
!pip -q install torchxrayvision

In [ ]:
import matplotlib.pyplot as plt
import os
import shutil
import torch
import torchxrayvision as xrv
import numpy as np
import pandas as pd
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [ ]:
# Short script that creates image folders by labels


# metadata = pd.read_csv("C:\\Users\\amitn\\Downloads\\covid-chestxray-dataset-master\\covid-chestxray-dataset-master\\metadata.csv")
# findings = metadata['finding'].unique() 
# for finding in findings:
#     filenames = metadata.loc[metadata['finding'] == finding]['filename']
#     finding = finding.replace('/','-')
#     if not os.path.exists("C:\\Users\\amitn\\Downloads\\covid-chestxray-dataset-master\\covid-chestxray-dataset-master\\images\\test\\" + finding):
#         os.makedirs("C:\\Users\\amitn\\Downloads\\covid-chestxray-dataset-master\\covid-chestxray-dataset-master\\images\\test\\" + finding)
#     for filename in filenames:
#         original = r'C:\\Users\\amitn\\Downloads\\covid-chestxray-dataset-master\\covid-chestxray-dataset-master\\images\\test\\' + filename
#         target = r'C:\\Users\\amitn\\Downloads\\covid-chestxray-dataset-master\\covid-chestxray-dataset-master\\images\\test\\' + finding
#         if os.path.exists(original):
#             shutil.move(original, target)

In [ ]:
data_dir = 'C:\\Users\\amitn\\Downloads\\covid-chestxray-dataset-master\\covid-chestxray-dataset-master\\images'

train_transforms = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                       transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.5), (0.5))])

test_transforms = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                     transforms.Resize(255),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor()])

train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [ ]:
model = xrv.models.DenseNet(weights="densenet121-res224-all")
model.classifier

Linear(in_features=1024, out_features=18, bias=True)

In [ ]:
# Don't backprop model parameters!
for param in model.parameters():
    param.requires_grad = False
    
# New classifier
model.classifier = nn.Sequential(nn.Linear(1024, 512),
                          nn.ReLU(),
                          nn.Dropout(0.2),
                          nn.Linear(512,256),
                          nn.ReLU(),
                          nn.Dropout(0.2),
                          nn.Linear(256,18),
                          nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()

# Training only the classifier parameters, model parameters remains unchanged
optimizer = optim.RMSprop(model.classifier.parameters(), lr=0.004)

# Move to GPU
model.to('cuda')

XRV-DenseNet121-densenet121-res224-all

In [ ]:
epochs = 3
for epoch in range(epochs):
    training_loss = 0
    for images, labels in trainloader:
        # Move to GPU
        images, labels = images.to('cuda'), labels.to('cuda')
        # Clear gradients
        optimizer.zero_grad()
        
        # Feed forward, calculating log probabilities
        logps = model(images)
        # Calculate loss, Negative log likelihood loss
        loss = criterion(logps, labels)

        # Backprop
        loss.backward()
        # Update weights
        optimizer.step()
        
        training_loss += loss.item()
    else:
        print(f"Training loss: {training_loss/len(trainloader)}")

Training loss: -0.4653121444086234
Training loss: -0.5838683247566223
Training loss: -0.5904939919710159


In [ ]:
# Testing
test_loss = 0
accuracy = 0
with torch.no_grad():
    model.eval()
    for images, labels in testloader:
        images, labels = images.to('cuda'), labels.to('cuda')
        # Feed forward test data
        logps = model(images)
        batch_loss = criterion(logps, labels)
        # Adding to total test loss
        test_loss += batch_loss
        
        # Calculate accuracy
        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        
    print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
    running_loss = 0
    model.train() 

Epoch 3/3.. Test loss: -0.551.. Test accuracy: 0.594
